# Jiu Yang (James) Wang

## Research question/interests

**My part of the project will be analyzing the correlation between housing prices and inventory shortfall as compared to population increases.**

My research question is looking at the correlation between housing prices and shortfall. Housing affordability is a pivotal issue for my generation, and i believe that the oft touted narrative of phantom overseas buyers is only a narrative meant to distract from the fact that the true cause of housing prices is a lack of new housing as compared to population. 

I plan to tackle my question by plotting housing price and invetory shortfall. I plan to calculate inventory shortfall by calculating the difference between population increases in regions and the amount of housing built. I will then present this to the reader by visualizing this data in graphs. I may also include the comparison to interest rates, and compare it to the shortfall plot. 

In [8]:
import pandas as pd

DailyInterestRate = pd.read_csv("../data/raw/DailyInterestRate.csv")
NewHousingPIndex = pd.read_csv("../data/raw/NewHousingPIndex.csv")
NewInventory = pd.read_csv("../data/raw/NewInventory.csv", low_memory = False)
PopEstimate = pd.read_csv("../data/raw/PopEstimate.csv")



In [41]:
#Pruning for interest rate table

#Prunes out data from before 2001-01-01
#Also selects only the target target rate
DailyInterestRatePruned = DailyInterestRate[(DailyInterstRate["Financial market statistics"] == "Target rate") & (DailyInterstRate["REF_DATE"] > "2001-01-01")] 
DailyInterestRatePruned2 = DailyInterestRatePruned[['REF_DATE', 'VALUE']]
DailyInterestRatePruned3 = DailyInterestRatePruned2.dropna()
#DailyInterestRatePruned3.sample(30)
DailyInterestRatePruned3.to_csv("../data/processed/DailyInterestRatePruned.csv")


In [54]:
#Creates list of locales

locales = NewHousingPIndex.GEO.unique()

#Creates a csv for each locale
for city in locales:
    name = city[:(city.find(",")+1) ]
    working = NewHousingPIndex[(NewHousingPIndex["GEO"] == city) & (NewHousingPIndex["REF_DATE"] > "2001-01-01")]
    working2 = working[['REF_DATE', 'VALUE']]
    working2.sample(10)
    name = "../data/processed/PIndex/" + name + "PIndex.csv"
    #print(name)
    working2.to_csv(name)
    
    




In [33]:
DailyInterestRate.sample(30)
#NewHousingPIndex.sample(10)
#locales = NewHousingPIndex.GEO.unique()


#NewInventory.sample(10)
#PopEstimate.sample(10)

,REF_DATE,GEO,DGUID,Financial market statistics,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
420345,2000-09-24,Canada,2016A000011124,"Bankers' acceptances rate, 1-month",Percent,239,units,0,v39068,1.29,NaN,..,NaN,t,2
410333,2000-01-05,Canada,2016A000011124,"Commercial paper, 3-month (composite rates)",Percent,239,units,0,v39049,1.11,5.27,NaN,NaN,t,2
533884,2012-03-28,Canada,2016A000011124,"Prime corporate paper rate, 1-month",Percent,239,units,0,v39072,1.32,1.07,NaN,NaN,t,2
622628,2021-03-28,Canada,2016A000011124,"Treasury Bills, 6-month",Percent,239,units,0,v39066,1.27,NaN,..,NaN,NaN,2
246528,1985-07-21,Canada,2016A000011124,"Bankers' acceptances rate, 2-month",Percent,239,units,0,v39070,1.30,NaN,..,NaN,t,2
343521,1995-03-14,Canada,2016A000011124,"Government of Canada Treasury Bills, 3-month (...",Percent,239,units,0,v39069,1.30,8.11,NaN,NaN,t,2
314111,1992-05-23,Canada,2016A000011124,"Treasury Bills, 1-month",Percent,239,units,0,v39063,1.24,NaN,..,NaN,NaN,2
41079,1964-09-20,Canada,2016A000011124,"Treasury Bills, 1-month",Percent,239,units,0,v39063,1.24,NaN,..,NaN,NaN,2
242355,1985-02-17,Canada,2016A000011124,"Government of Canada benchmark bond yields, 5 ...",Percent,239,units,0,v39053,1.15,NaN,..,NaN,NaN,2
342356,1995-02-11,Canada,2016A000011124,"Government of Canada benchmark bond yields, 7 ...",Percent,239,units,0,v39054,1.16,NaN,..,NaN,NaN,2
